# Классы в Python

In [1]:
class Foo(object):
    def __init__(self, x=1, y=2):
        self.x = x
        self.y = y
        print("Foo")
    
    def __my_method(self, z):
        print(z)
        
    def f(self, z):
        self.__my_method(z)

foo = Foo()
print(foo.x, foo.y)
foo.f(123)

Foo
1 2
123


In [3]:
foo.__my_method(123)

AttributeError: 'Foo' object has no attribute '__my_method'

In [4]:
print(foo)

In [6]:
print(type(foo))

<class '__main__.Foo'>


In [7]:
class MyLinReg(object):
    def __init__(self, C=0.01, regularizer='l2'):
        self.C = C
        self.regularizer = regularizer
        
    def fit(self, X, y):
        self.w = np.dot(X.T, np.dot( (np.dot(X, X.T)), y ) )
        
    def predict(self, X):
        return X.dot(w)

# Декораторы в Python

In [9]:
import time


def timer(f):
    def tmp(*args, **kwargs):
        t = time.time()
        res = f(*args, **kwargs)
        print("Время выполнения функции %s: %f" % (str(f), (time.time()-t)))
        return res

    return tmp


def memoized(f):
    memory = {}

    def wrapper(*args, **kwargs):
        key = (tuple(args), hash(tuple(sorted(kwargs.items()))))
        if not key in memory:
            memory[key] = f(*args, **kwargs)
        return memory[key]
    return wrapper

In [15]:
@timer
@memoized
def sqr(x):
    return x ** 190000

#sqr = timer(sqr)

for k in range(20):
    sqr(3 ** 7)

Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.093541
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper at 0x000001B4266B5A60>: 0.000000
Время выполнения функции <function memoized.<locals>.wrapper

In [19]:
class Foo:
    def __init__(self):
        print("Foo")

    @property
    def x(self):
        return 1
    
foo = Foo()
print(Foo.x)
print(foo.x)

Foo
1


# Beautiful Soup, urllib и chardet: парсим веб-страницы

In [ ]:
from collections import deque
import codecs
import os
import time
import urllib2
import chardet

site = "http://simple.wikipedia.org"

def read_url(name):
    html = urllib2.urlopen(name).read()
    encoding = chardet.detect(html)['encoding']
    if encoding != 'unicode':
        html = html.decode(encoding)
    return html


def parse_url(name):
    html = read_url(name)
    soup = bs4.BeautifulSoup(html)

    wiki = "/wiki/"

    return [x for x in [hit.attrs.get("href","") for hit in soup.findAll('a')] if x[:len(wiki)] == wiki]

def save_html(link, link_id, docs_folder):
        html = read_url(link)
        out_file = codecs.open(os.path.join(docs_folder, str(link_id) + ".html"), "w", "utf-8")
        print >> out_file, html


def crawl(start_link="/wiki/Main_Page"):
    queue = deque([start_link])
    id_queue = deque([0])

    cur_id = 0

    visited = dict()

    links_id_map = codecs.open("link_id_map.txt", "w", "utf-8")
    graph = codecs.open("graph.txt", "w", "utf-8")
    visited[start_link] = 0

    while queue:
        link = site + queue.popleft()
        print link
        link_id = id_queue.popleft()

        try:
            try:
                save_html(link, link_id, "./docs/")
            except:
                time.sleep(60)
                save_html(link, link_id, "./docs/")

            print >> links_id_map, link_id, link


            links = parse_url(link)

            links_ids = []
            for lnk in links:
                if lnk in visited.keys():
                    links_ids.append(visited[lnk])
                else:
                    cur_id += 1
                    visited[lnk] = cur_id
                    queue.append(lnk)
                    id_queue.append(cur_id)

            print >> graph, str(link_id) + ":" + ",".join([str(i) for i in links_ids])
        except:
            pass